In [ ]:
lam = 0.01
TX = np.array([[0, 0], [0.005, 0], [0.01, 0], [0.015, 0]])
RX_2 = list(np.array([-10, -40]) + 0.005*np.array([np.cos(math.radians(150)), np.sin(math.radians(150))]))
RX = np.array([[-10, -40], RX_2])
geo = geometry(TX, RX, lam, 0.005, scatterer = np.array([0, 0]), random_phase = math.pi/6)
BT = np.zeros((4, 2), dtype=np.complex128)
BR = np.zeros((2, 2), dtype=np.complex128)
A = np.zeros((2, 2), dtype=np.complex128)
BT[:, 0] = geo.calculate_bT().reshape(-1, )
BR[:, 0] = geo.calculate_bR().reshape(-1, )
A[0, 0] = geo.calculate_a()
geo = geometry(TX, RX, lam, 0.005, scatterer = np.array([20, -25]), random_phase = math.pi/6)
BT[:, 1] = geo.calculate_bT().reshape(-1, )
BR[:, 1] = geo.calculate_bR().reshape(-1, )
A[1, 1] = geo.calculate_a()


H = BR@A@BT.conj().T
H_partial = BR@np.absolute(A)@BT.conj().T

He = BR.conj().T@H_partial@BT
omega = scipy.linalg.inv(scipy.linalg.sqrtm((BR.conj().T)@BR))
ff = scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT))
He2 = omega@He@ff

He_p = BR.conj().T@H@BT
omega = scipy.linalg.inv(scipy.linalg.sqrtm((BR.conj().T)@BR))
ff = scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT))
He3 = omega@He_p@ff
# He2 = BR.conj().T@H@BT



rates1_CSIT_nobeam = []
rates1_pCSIT_nobeam = []
rates1_CSIT_beam = []
rates1_pCSIT_beam = []
for p in np.arange(0, 65, 5):

  #CSIT_nobeam
  PT = 10**(p/10)
  L = 1
  chan_mat = np.zeros((L, L), dtype = object)
  chan_mat[0, 0] = H
  w = list(np.ones(L, dtype = int))
  ss = init_sigma(L, 4, PT)
  _, r, _, sigma, _ = alg_dual_link(PT, w, chan_mat, epsilon, Sigma = ss)
  rates1_CSIT_nobeam.append(r)

  #pCSIT_nobeam
  PT = 10**(p/10)
  L = 1
  chan_mat = np.zeros((L, L), dtype = object)
  chan_mat[0, 0] = H_partial
  w = list(np.ones(L, dtype = int))
  ss = init_sigma(L, 4, PT)
  _, _, _, sigma, _ = alg_dual_link(PT, w, chan_mat, epsilon, Sigma = ss)
  r = np.log2(np.linalg.det(np.identity(2)+H@sigma[0]@(H.conj().T)))
  rates1_pCSIT_nobeam.append(r)

  #pCSIT_beam
  PT = 10**(p/10)
  L = 1
  chan_mat = np.zeros((L, L), dtype = object)
  chan_mat[0, 0] = He2
  w = list(np.ones(L, dtype = int))
  ss = init_sigma(L, 2, PT)
  _, _, _, sigma, _ = alg_dual_link(PT, w, chan_mat, epsilon, Sigma = ss)
  ss = (scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT)))@(sigma[0])@(scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT)))
  Sig = BT@ss@BT.conj().T
  r = np.log2(np.linalg.det(np.identity(2)+H@Sig@(H.conj().T)))
  rates1_pCSIT_beam.append(r)

  #CSIT_beam
  PT = 10**(p/10)
  L = 1
  chan_mat = np.zeros((L, L), dtype = object)
  chan_mat[0, 0] = He3
  w = list(np.ones(L, dtype = int))
  ss = init_sigma(L, 2, PT)
  _, _, _, sigma, _ = alg_dual_link(PT, w, chan_mat, epsilon, Sigma = ss)
  ss = (scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT)))@(sigma[0])@(scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT)))
  Sig = BT@ss@BT.conj().T
  r = np.log2(np.linalg.det(np.identity(2)+H@Sig@(H.conj().T)))
  rates1_CSIT_beam.append(r)


In [ ]:
# F_I_0 = np.hstack((BTT[0, 0], BTT[1, 0]))
# F_I_1 = np.hstack((BTT[1, 1], BTT[0, 1]))
# F_IR_0 = np.hstack((BRT[0, 0], BRT[0, 1]))
# F_IR_1 = np.hstack((BRT[1, 1], BRT[1, 0]))

F_I_0 = BTT[0, 0]
F_I_1 = BTT[1, 1]
F_IR_0 = BRT[0, 0]
F_IR_1 = BRT[1, 1]

HeT[0, 0] = F_IR_0.conj().T@BRT[0, 0]@np.absolute(AT[0, 0])@BTT[0, 0].conj().T@F_I_0
HeT[0, 1] = F_IR_0.conj().T@BRT[0, 1]@np.absolute(AT[0, 1])@BTT[0, 1].conj().T@F_I_1
HeT[1, 0] = F_IR_1.conj().T@BRT[1, 0]@np.absolute(AT[1, 0])@BTT[1, 0].conj().T@F_I_0
HeT[1, 1] = F_IR_1.conj().T@BRT[1, 1]@np.absolute(AT[1, 1])@BTT[1, 1].conj().T@F_I_1

HeT2[0, 1] = (scipy.linalg.inv(scipy.linalg.sqrtm(F_IR_0.conj().T@F_IR_0)))@HeT[0, 1]@(scipy.linalg.inv(scipy.linalg.sqrtm(F_I_1.conj().T@F_I_1)))
HeT2[1, 0] = (scipy.linalg.inv(scipy.linalg.sqrtm(F_IR_1.conj().T@F_IR_1)))@HeT[1, 0]@(scipy.linalg.inv(scipy.linalg.sqrtm(F_I_0.conj().T@F_I_0)))
HeT2[0, 0] = (scipy.linalg.inv(scipy.linalg.sqrtm(F_IR_0.conj().T@F_IR_0)))@HeT[0, 0]@(scipy.linalg.inv(scipy.linalg.sqrtm(F_I_0.conj().T@F_I_0)))
HeT2[1, 1] = (scipy.linalg.inv(scipy.linalg.sqrtm(F_IR_1.conj().T@F_IR_1)))@HeT[1, 1]@(scipy.linalg.inv(scipy.linalg.sqrtm(F_I_1.conj().T@F_I_1)))

# HeT[0, 0] = BRT[0, 0]@np.absolute(AT[0, 0])@BTT[0, 0].conj().T@F_I_0
# HeT[0, 1] = BRT[0, 1]@np.absolute(AT[0, 1])@BTT[0, 1].conj().T@F_I_1
# HeT[1, 0] = BRT[1, 0]@np.absolute(AT[1, 0])@BTT[1, 0].conj().T@F_I_0
# HeT[1, 1] = BRT[1, 1]@np.absolute(AT[1, 1])@BTT[1, 1].conj().T@F_I_1

# HeT2[0, 1] = HeT[0, 1]@(scipy.linalg.inv(scipy.linalg.sqrtm(F_I_1.conj().T@F_I_1)))
# HeT2[1, 0] = HeT[1, 0]@(scipy.linalg.inv(scipy.linalg.sqrtm(F_I_0.conj().T@F_I_0)))
# HeT2[0, 0] = HeT[0, 0]@(scipy.linalg.inv(scipy.linalg.sqrtm(F_I_0.conj().T@F_I_0)))
# HeT2[1, 1] = HeT[1, 1]@(scipy.linalg.inv(scipy.linalg.sqrtm(F_I_1.conj().T@F_I_1)))

rates1_pCSIT_narrowbeam = []
sigmas_pCSIT_optbeam = []
for p in np.arange(snr_min, snr_max, 5):

  print(p)
  PT = 10**(p/10)
  gain = 10
  L = 2
  if p == 0:
    rate2l = []
    sigmas2l = []
    for i in range(n_inits):
      w = list(np.ones(L, dtype = int))
      ss = init_sigma(L, 2, PT)
      chan_mat = HeT2
      _, r_beam, _, sigma_beam, _ = alg_dual_link(PT, w, chan_mat, epsilon, Sigma = ss)
      rate2l.append(r_beam)
      sigmas2l.append(sigma_beam)
    sigma2_new = sigmas2l[rate2l.index(max(rate2l))]
    ss0 = (scipy.linalg.inv(scipy.linalg.sqrtm((F_I_0.conj().T)@F_I_0)))@(sigma2_new[0])@(scipy.linalg.inv(scipy.linalg.sqrtm((F_I_0.conj().T)@F_I_0)))
    ss1 = (scipy.linalg.inv(scipy.linalg.sqrtm((F_I_1.conj().T)@F_I_1)))@(sigma2_new[1])@(scipy.linalg.inv(scipy.linalg.sqrtm((F_I_1.conj().T)@F_I_1)))
    Sig0 = F_I_0@ss0@F_I_0.conj().T
    Sig1 = F_I_1@ss1@F_I_1.conj().T
    om0 = np.identity(2) + HT[0, 1]@Sig1@HT[0, 1].conj().T
    om1 = np.identity(2) + HT[1, 0]@Sig0@HT[1, 0].conj().T
    I0 = np.log2(np.linalg.det(np.identity(2)+HT[0, 0]@Sig0@(HT[0, 0].conj().T)@(np.linalg.inv(om0))))
    I1 = np.log2(np.linalg.det(np.identity(2)+HT[1, 1]@Sig1@(HT[1, 1].conj().T)@(np.linalg.inv(om1))))
    rate2 = I0+I1
    s4 = np.zeros(2, dtype = object)
    s4[0] = Sig0
    s4[1] = Sig1

  else:
    rate2l = []
    sigmas2l = []
    w = list(np.ones(L, dtype = int))
    sss = sigma2_new
    chan_mat = HeT2
    om0 = np.identity(2) + HeT2[0, 1]@sss[1]@HeT2[0, 1].conj().T
    om1 = np.identity(2) + HeT2[1, 0]@sss[0]@HeT2[1, 0].conj().T
    I0 = np.log2(np.linalg.det(np.identity(2)+HeT2[0, 0]@sss[0]@(HeT2[0, 0].conj().T)@(np.linalg.inv(om0))))
    I1 = np.log2(np.linalg.det(np.identity(2)+HeT2[1, 1]@sss[1]@(HeT2[1, 1].conj().T)@(np.linalg.inv(om1))))
    rate2l.append(I0+I1)
    sigmas2l.append(sigma2_new)
    for i in range(n_inits):
      w = list(np.ones(L, dtype = int))
      ss = init_sigma(L, 2, PT)
      chan_mat = HeT2
      _, r_beam, _, sigma_beam, _ = alg_dual_link(PT, w, chan_mat, epsilon, Sigma = ss)
      rate2l.append(r_beam)
      sigmas2l.append(sigma_beam)
    sigma2_new = sigmas2l[rate2l.index(max(rate2l))]
    ss0 = (scipy.linalg.inv(scipy.linalg.sqrtm((F_I_0.conj().T)@F_I_0)))@(sigma2_new[0])@(scipy.linalg.inv(scipy.linalg.sqrtm((F_I_0.conj().T)@F_I_0)))
    ss1 = (scipy.linalg.inv(scipy.linalg.sqrtm((F_I_1.conj().T)@F_I_1)))@(sigma2_new[1])@(scipy.linalg.inv(scipy.linalg.sqrtm((F_I_1.conj().T)@F_I_1)))
    Sig0 = F_I_0@ss0@F_I_0.conj().T
    Sig1 = F_I_1@ss1@F_I_1.conj().T
    om0 = np.identity(2) + HT[0, 1]@Sig1@HT[0, 1].conj().T
    om1 = np.identity(2) + HT[1, 0]@Sig0@HT[1, 0].conj().T
    I0 = np.log2(np.linalg.det(np.identity(2)+HT[0, 0]@Sig0@(HT[0, 0].conj().T)@(np.linalg.inv(om0))))
    I1 = np.log2(np.linalg.det(np.identity(2)+HT[1, 1]@Sig1@(HT[1, 1].conj().T)@(np.linalg.inv(om1))))
    rate2 = I0+I1
    s4 = np.zeros(2, dtype = object)
    s4[0] = Sig0
    s4[1] = Sig1

  sigmas_pCSIT_optbeam.append(s4)
  rates1_pCSIT_narrowbeam.append(rate2)
